In [ ]:
import numpy as np
import pandas as pd
import re
import requests
import lxml
from bs4 import BeautifulSoup as bs
from tqdm import tqdm
import time
import config

In [ ]:
sex = "男歌手"
df_singers = pd.read_csv(rf"../data/{sex}.csv")
df_singers

,歌手,链接
0,林俊杰,https://music.163.com/artist?id=3684
1,陈奕迅,https://music.163.com/artist?id=2116
2,郑润泽,https://music.163.com/artist?id=29051613
3,颜人中,https://music.163.com/artist?id=31376161
4,汪苏泷,https://music.163.com/artist?id=5538
...,...,...
95,THOME,https://music.163.com/artist?id=12110173
96,刘思达LOFTHESTAR,https://music.163.com/artist?id=12030367
97,杨和苏KeyNG,https://music.163.com/artist?id=13899303
98,MC HotDog 热狗,https://music.163.com/artist?id=4479


In [ ]:
# dict_songs = dict()
df_songs = pd.DataFrame({"singer": [], "song_name": [], "song_link": []})
tqdm.pandas()


def get_songs(row):
    response = requests.get(
        url=row.链接, headers=config.HEADERS, params={}, cookies=config.COOKIES
    )
    soup = bs(response.text, "lxml")
    # print(soup.prettify())
    # pbar = tqdm.get_lock().get_instances()[0]
    # pbar.set_description_str(f'processing {row.歌手}')

    song_names = [
        song.text for song in soup.find_all("a", href=re.compile(r"/song\?id=\d"))
    ]
    song_links = [
        r"https://music.163.com" + song["href"]
        for song in soup.find_all("a", href=re.compile(r"/song\?id=\d"))
    ]

    global df_songs
    df_songs = pd.concat(
        [
            df_songs,
            pd.DataFrame(
                {
                    "singer": [row.歌手 for _ in range(len(song_names))],
                    "song_name": song_names,
                    "song_link": song_links,
                }
            ),
        ],
        ignore_index=True,
    )
    # return df_songs_new
    # time.sleep(2.0)

In [54]:
df_singers.progress_apply(get_songs, axis=1)

100%|██████████| 100/100 [01:04<00:00,  1.54it/s]


0     None
1     None
2     None
3     None
4     None
      ... 
95    None
96    None
97    None
98    None
99    None
Length: 100, dtype: object

In [ ]:
df_songs.to_csv(rf"../data/{sex}歌曲.csv", header=True, index=False)